# Map georeferencing notebook - Python

In [ ]:
import urllib.request
import pandas as pd
import geopandas
import geopy
from geopy.geocoders import Nominatim
import os
import folium
import math
from folium import Choropleth, Circle, Marker
from folium.plugins import MarkerCluster
import geopandas as gpd
import numpy as np

import datetime
import locale

In [ ]:
# url = "https://raw.githubusercontent.com/AngelusGi/UniMolFinalExam/main/Dataset/Original_CSV/Angelo/trackLog-2019-dic-23_13-53-06.csv"
# file_name = "trackLog-2019-dic-23_13-53-06.csv"

In [ ]:
# urllib.request.urlretrieve(url, filename=file_name)

Questo metodo elimina dal corrente dataframe tutte le colonne, eccezion fatta per latitudine e longitudine

In [ ]:
def clean_header(file_to_process):

    dataTypeDict = {"GPS Time":str ,"Device Time":str ,"Longitude":float, "Latitude":float, "GPS Speed (Meters/second)":float, "Altitude":float, "G(x)":float, "G(y)":float, "G(z)":float, "Kilometers Per Litre(Instant)(kpl)": float, "Litres Per 100 Kilometer(Instant)(l/100km)":float, "Actual engine % torque(%)":float, "Trip Distance(km)":float, "Engine RPM(rpm)":float, "GPS Accuracy(m)":float, "GPS Altitude(m)":float, "GPS Latitude(°)":float, "GPS Longitude(°)":float, "Acceleration Sensor(X axis)(g)":float, "Acceleration Sensor(Y axis)(g)":float, "Acceleration Sensor(Z axis)(g)":float, "Acceleration Sensor(Total)(g)":float, "Run time since engine start(s)":int, "Driver":str}

    df = pd.read_csv(file_to_process, dtype=dataTypeDict)
    # keeps_only_latidue_longitude = [ "GPS Time","Device Time","GPS Speed (Meters/second)","Altitude","G(x)","G(y)","G(z)","Kilometers Per Litre(Instant)(kpl)","Litres Per 100 Kilometer(Instant)(l/100km)","Actual engine % torque(%)","Trip Distance(km)","Engine RPM(rpm)","GPS Accuracy(m)","GPS Altitude(m)","GPS Latitude(°)","GPS Longitude(°)","Acceleration Sensor(X axis)(g)","Acceleration Sensor(Y axis)(g)","Acceleration Sensor(Z axis)(g)","Acceleration Sensor(Total)(g)","Run time since engine start(s)","Driver"]
    keeps_only_latidue_longitude_time = [ "GPS Time","GPS Speed (Meters/second)","Altitude","G(x)","G(y)","G(z)","Kilometers Per Litre(Instant)(kpl)","Litres Per 100 Kilometer(Instant)(l/100km)","Actual engine % torque(%)","Trip Distance(km)","Engine RPM(rpm)","GPS Accuracy(m)","GPS Altitude(m)","GPS Latitude(°)","GPS Longitude(°)","Acceleration Sensor(X axis)(g)","Acceleration Sensor(Y axis)(g)","Acceleration Sensor(Z axis)(g)","Acceleration Sensor(Total)(g)","Run time since engine start(s)","Driver"]

    # gps_long_lat = df.drop(columns=keeps_only_latidue_longitude)    
    # return gps_long_lat

    gps_long_lat_time = df.drop(columns=keeps_only_latidue_longitude_time)
    return gps_long_lat_time

In [ ]:
def add_csv_to_df(file_to_process):
    return pd.read_csv(file_to_process)

codice per la gestione delle time stanp nel csv

In [ ]:
def ParseGpsTime(gps_value:str):
    local_setting_gps_time = 'en_US'
    locale.setlocale(locale.LC_ALL, local_setting_gps_time)

    gps_format = '%a %b %d %H:%M:%S %z %Y'
    # gps_test = "Mon Dec 30 09:16:50 +0100 2019"

    date_time_gps = datetime.datetime.strptime(gps_value, gps_format)

    # print("GPS:")
    # print('Date:', date_time_gps.date())
    # print('Time:', date_time_gps.time())
    # print('Date-time:', date_time_gps)

    return date_time_gps

In [ ]:
def ParseDeviceTime(device_value:str):
    local_setting_device_time = 'it_IT'
    locale.setlocale(locale.LC_ALL, local_setting_device_time)

    device_format = '%d-%b-%Y %H:%M:%S.%f'
    # device_time = "30-dic-2019 09:16:52.347"

    date_time_device = datetime.datetime.strptime(device_value, device_format)

    # print("Device:")
    # print('Date:', date_time_device.date())
    # print('Time:', date_time_device.time())
    # print('Date-time:', date_time_device)

    return date_time_device

prede tutti i file nella cartella e li trasforma in un dizionario di dataframe

In [ ]:
# verificare csv_path
basepath = os.environ['USERPROFILE']
githubpath = r"\GitHub\UniMolFinalExam"

# csv_path = r"{}{}\Notebooks\Python\TestSet".format(basepath,githubpath)
csv_path = r"{}{}\Dataset\Final".format(basepath,githubpath)

In [ ]:
coordinates_data = dict()
full_data = dict()

In [ ]:
for directory, subdir_list, file_list in os.walk(csv_path):
    for file_name in file_list:
        print('File:', file_name)
        path_to_read_csv = csv_path + "\\" + file_name
        print(path_to_read_csv)

        temp_dataframe = clean_header(path_to_read_csv)
        temp_file_name_ext = file_name.replace('.csv','')
        final_file_name = temp_file_name_ext.replace('trackLog-','')
        coordinates_data[final_file_name] = temp_dataframe

        full_data[final_file_name] = add_csv_to_df(path_to_read_csv)
        print()
    print()

crea un dizionario contenete solo le coordinate di latitudine e longitudine

In [ ]:
coordinates_data

In [ ]:
coordinates_data["Test"].head()

estrae un singolo dataframe

In [ ]:
gps_long_lat = coordinates_data["Test"]

In [ ]:
gps_long_lat

ottenere un indirizzo a partire dalle coordinate

In [ ]:
locator = Nominatim(user_agent="agvGeoCoder")

In [ ]:
# gps_address = locator.geocode("42.048168,13.931077")

In [ ]:
coordinate = gps_long_lat.iloc[0]

In [ ]:
gps_address = locator.geocode("{},{}".format(coordinate["Latitude"],coordinate["Longitude"]))

In [ ]:
print("Location = {}".format(gps_address))
gps_address.point

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="UniMolFinalExam")

In [ ]:
place_to_find = "Italy"
geo_cordinates = geolocator.geocode(place_to_find)
geo_cordinates

In [ ]:
# map_dimension = 650
# map_zoom = 6

# italy_map = folium.Map(location=[geo_cordinates.latitude,geo_cordinates.longitude], zoom_start=map_zoom, width=map_dimension, height=map_dimension)

In [ ]:
gps_address

In [ ]:
gps_address.latitude

In [ ]:
italy_map = folium.Map(location=[gps_address.latitude,gps_address.longitude])

In [ ]:
# italy_map

In [ ]:
for index in range(len(gps_long_lat)):
    position = gps_long_lat.iloc[index]
    
    folium.Circle(
      location=[position["Latitude"],position["Longitude"]],
      radius = 20,
      popup= "test",
      color= 'crimson',
      fill= True,
      fill_color = 'crimson'
   ).add_to(italy_map)

In [ ]:
italy_map

esporta la mappa prodotta in un file html

In [ ]:
italy_map.save("TestMap.html")